In [2]:
import pandas as pd
import numpy as np

from os import listdir

In [6]:
data = pd.DataFrame(columns=["File","Vendor","Invoice Number","Amount","Invoice Date"])
folder = "./Results/"
files = listdir(folder)
for file in files:
    path = folder + file
    data_new = pd.read_csv(path, sep=';', header=None)
    data_new.columns = ["File","Vendor","Invoice Number","Amount","Invoice Date"]
    #print(data_new.dtypes)
    data = data._append(data_new,ignore_index=True)
data.head()

,File,Vendor,Invoice Number,Amount,Invoice Date
0,0,NaN,NaN,440,NaN
1,B24_Rechnung 50046676,BIKE24 Bike24 GmbH Bike24 GmbH,50046676,NaN,04/24/2021 00:00:000


In [33]:
data.dtypes


File              object
Vendor            object
Invoice Number    object
Amount            object
Invoice Data      object
dtype: object